# Virtualization: hypervisors; containers
_COSC 208, Introduction to Computer Systems, Spring 2025_

## Virtual machines (VMs)

* Recall: _what is the purpose of an OS?_
    * Make computer hardware easy to use—e.g., an OS knows how to load an application's executable code from persistent storage (e.g., solid state drive (SSD)) into main memory, initialize the process's memory (code, heap, stack), and make the CPU execute the application's instructions
    * Support multiprocessing—i.e., running multiple applications simultaneously or concurrently
    * Allocate and manage hardware resources—e.g., decide when/which applications can use the CPU, decide when/which memory applications can use, prevent applications from stealing/accessing another application's CPU time or memory
* What is the purpose of a virtual machine?
    * Make computer hardware flexible to use
    * Support multiple OSes running concurrently
    * Allocate and manage hardware resources—e.g., decide when/which VMs can use the CPU, decide/when which memory VMs can use, prevent VMS from stealing/accessing another VM's CPU time or memory
* What is the motivation for using a virtual machine?
    * Run multiple OSes concurrently — _why would someone one to do this?_
        * Run software only supported by a specific OS
        * Software testing
        * Server consolidation
        * Security sandboxing
    * Flexibly use hardware resources

<p style="height:15em;"></p>

## Hypervisors

* Hypervisor – an additional software layer a computer system needs to support virutal machines
* Where does a hypervisor reside?
    * System that does **not** use a hypervisor
        ```
        +----+ +----+
        |Proc| |Proc|
        +----+ +----+
        +-----------+
        |    OS     |
        +-----------+
        +-----------+
        |  Hardware |
        +-----------+
        ```
    * Type 1: on top of hardware — e.g., KVM, Xen, Hyper-V, VMware vSphere
        ```
        +-------------+ +-------------+
        |     VM      | |     VM      |
        |+----+ +----+| |+----+ +----+|
        ||Proc| |Proc|| ||Proc| |Proc||
        |+----+ +----+| |+----+ +----+|
        |+-----------+| |+-----------+|
        ||    OS     || ||    OS     ||
        |+-----------+| |+-----------+|
        +-------------+ +-------------+
        +-----------------------------+
        |          Hypervisor         |
        +-----------------------------+
        +-----------------------------+
        |           Hardware          |
        +-----------------------------+
        ```
    * Type 2: on top of host OS — e.g., VirtualBox, VMware Workstation/Fusion
        ```
                      +-------------+ +-------------+
                      |     VM      | |     VM      |
                      |+----+ +----+| |+----+ +----+|
                      ||Proc| |Proc|| ||Proc| |Proc||
                      |+----+ +----+| |+----+ +----+|
                      |+-----------+| |+-----------+|
                      || Guest OS  || || Guest OS  ||
                      |+-----------+| |+-----------+|
                      +-------------+ +-------------+
        +----+ +----+ +-----------------------------+
        |Proc| |Proc| |          Hypervisor         |
        +----+ +----+ +-----------------------------+
        +-------------------------------------------+
        |                 Host OS                   |
        +-------------------------------------------+
        +-------------------------------------------+
        |                Hardware                   |
        +-------------------------------------------+
        ```
* Why is a type 1 hypervisor better than a type 2 hypervisor?
    * Better performance – no overhead from host OS
* Why is a type 2 hypervisor better than a type 1 hypervisor?
    * Easier to use – runs alongside other applications on an existing computer system

<p style="height:25em;"></p>

## CPU sharing

* Use same approach OS uses for sharing CPU between processes — Limited Direct Execution (LDE)
* _Recall: How does LDE work in OS?_
    * Put CPU in user mode
    * Give process exclusive access to CPU
    * Process issues a syscall to perform privileged operation
    * Timer interrupt allows OS to regain control
* To use LDE with a VM
    * Put CPU in user mode
    * Give VM exclusive access to CPU
    * Process issues a syscall -OR- OS attempts to perform privileged operation
    * Timer interrupt allows hypervisor to regain control
* Recall: _How does a process make a syscall?_ — load syscall number into register `x8` and execute `svc` instruction
* _What happens when a process makes a syscall?_
    1. CPU switches to kernel mode
    2. CPU saves registers to the OS stack
    3. CPU uses system call number to index into OS's table of syscall handlers 
    4. Branch and link to syscall handler code
    5. Restore registers from the OS stack
    6. Switch CPU to user mode
    7. Resume execution after `svc` 
* Syscalls with a type 1 hypervisor
    * Processes in virtualized OS do the same as before
    * Hypervisor’s syscall handler is invoked by hardware instead of OS’s syscall handler
    * Actually want to invoke OS's syscall handler — the hypervisor does this
    * Hypervisor knows where OS trap table is located, because populating syscall table is a privileged instruction, so hypervisor interposed on these operations when OS booted
* Steps when process in virtualized OS makes a syscall
    1. CPU switches to supervisor mode
    2. CPU saves registers to the hypervisor stack
    3. CPU uses system call number to index into VM's table of syscall handlers 
    4. **Switch CPU from supervisor to kernel mode**
    5. Branch and link to syscall handler code
    6. **Switch CPU from kernel to supervisor mode**
    7. Restore registers from the hypervisor stack
    8. Switch CPU to user mode
    9. Resume execution after `svc`
* Extra work to always trap to hypervisor, which then invokes OS code
* Possible solutions
    * Hardware support — e.g., supervisor mode that is between user and kernel mode
    * Para-virtualization — modify the OS so it's aware there is a hypervisor and the two can interact directly
    * Containers — use a single OS, but create groups of processes that are isolated from other processes
        * OS's process abstraction already isolates each process's CPU time, memory, and registers, but processes can still use syscalls to pass messages to each other, wait for each other, access a common set of files, etc.

## Containers

* Containers — use a single OS, but create groups of processes that are isolated from other processes
    * OS's process abstraction already isolates each process's CPU time, memory, and registers, but processes can still use syscalls to pass messages to each other, wait for each other, access a common set of files, etc.
* Containers share the kernel — i.e., core OS functionality like syscalls, scheduler, I/O interface
    * VMs do **not** share the kernel
* Containers and VMs do **not** share the file system ==> containers and virtual machines do **not** share libraries, executables, etc.
* Containers and VMs do **not** share user accounts
* Containers and VMs do **not** share network stacks ==> e.g., each container or VM has its own Internet Protocol (IP) address
* Containers and VMs share the hardware

<p style="height:10em;"></p>

## Recap

Q1: _What is an advantage of using **virtual machines with a type 1 hypervisor** instead of **virtual machines with a type 2 hypervisor**?_

With a type 1 hypervisor, system calls issued by processes only need to be handled by the hypervisor and guest operating system, whereas with a type 2 hypervisor, system calls issued by processes need to be handled by the host operating system, hypervisor, and guest operating system. Thus, a type 1 hypervisor adds less overhead and results in better performance.

<p style="height:8em;"></p>

Q2: _What is an advantage of using **virtual machines with a type 2 hypervisor** instead of **virtual machines with a type 1 hypervisor**?_

A type 2 hypervisor can be installed atop an existing (host) operating system just like any other application, whereas a type 1 hypervisor needs to be installed directly atop the hardware before any operating systems are installed. Thus, a type 2 hypervisor is easier to set up.

<p style="height:8em;"></p>

Q3: _What is an advantage of using **containers** instead of **virtual machines with a type 1 hypervisor**?_

With containers, all processes run atop a single operating system instances, whereas with a type 1 hypervisor, each group of processes requires a separate operating system. Thus, containers avoid the need to maintain multiple copies of the same operating system.

With containers, systems calls issued by processes only need to be handled by the operating system, just like a system without containers or virtual machines, whereas with a type 1 hypervisor, system calls issued by processes need to be handled by the hypervisor and the operating system running inside the virtual machine. Thus, containers add less overhead and result in better performance.

<p style="height:8em;"></p>

Q4: _What is an advantage of using **virtual machines with a type 1 hypervisor** instead of **containers**?_

With a type 1 hypervisor, multiple flavors (e.g., Windows, Linux, Mac OS) and/or versions (e.g., Windows 8, Windows 10) of operating systems can run atop the same physical hardware, whereas with containers only a single operating system can run atop the hardware. Thus, type 1 hypervisors allow multiple OSes to be run on the same computer system.